# < Phase 4: 도메인 특화 피처 생성 >
- 선수별 패스 스타일 (4개)
- 팀별 전술 성향 (2개)  
- 경기 흐름 (3개)

**총 9개 피처 추가**

## 1. 환경설정 & Data loading

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path

In [2]:
# 한글 폰트 설정 (Windows)
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

# 경로 설정
DATA_DIR = Path('../data/raw')
PROCESSED_DIR = Path('../data/processed')

In [3]:
# 데이터 파일 확인 및 크기 출력
print("📂 사용 가능한 데이터 파일:")
for file in DATA_DIR.glob('*.csv'):
    size = file.stat().st_size / 1024 / 1024  # MB
    print(f"  - {file.name}: {size:.2f} MB")

📂 사용 가능한 데이터 파일:
  - match_info.csv: 0.03 MB
  - sample_submission.csv: 0.05 MB
  - test.csv: 0.10 MB
  - train.csv: 41.54 MB


In [4]:
print("=" * 50)
print("Phase 4: 도메인 특화 피처 생성")
print("=" * 50)

# 데이터 불러오기
# 기존 Phase 3 데이터
df_v3 = pd.read_csv(PROCESSED_DIR / 'train_final_passes_v3.csv')
print(f"\n✅ Phase 3 데이터 로드: {df_v3.shape}")

# 원본 train.csv (전체 액션)
train_full = pd.read_csv(DATA_DIR / 'train.csv')
print(f"✅ 원본 train 로드: {train_full.shape}")

# 경기 정보
match_info = pd.read_csv(DATA_DIR / 'match_info.csv')
print(f"✅ match_info 로드: {match_info.shape}")

Phase 4: 도메인 특화 피처 생성

✅ Phase 3 데이터 로드: (15435, 43)
✅ 원본 train 로드: (356721, 15)
✅ match_info 로드: (228, 17)


## 2. 데이터 전처리

In [5]:
print("\n" + "=" * 60)
print("player_id, team_id 추가")
print("=" * 60)

# train.csv에서 최종 패스의 정보 가져오기
final_passes_info = train_full[train_full['type_name'] == 'Pass'].copy()
final_passes_info = final_passes_info.sort_values('time_seconds')
final_passes_info = final_passes_info.groupby('game_episode').last().reset_index()
final_passes_info = final_passes_info[['game_episode', 'game_id', 'team_id', 'player_id', 'is_home']]

# merge
df_v3 = df_v3.merge(final_passes_info, on='game_episode', how='left', suffixes=('', '_final'))
if 'game_id_final' in df_v3.columns:
    df_v3 = df_v3.drop(columns=['game_id_final'])

print(f"✅ player_id, team_id 추가 완료")
print(f"   결측치: player_id={df_v3['player_id'].isna().sum()}, team_id={df_v3['team_id'].isna().sum()}")


player_id, team_id 추가
✅ player_id, team_id 추가 완료
   결측치: player_id=0, team_id=0


In [6]:
print("\n" + "=" * 60)
print("Pass 데이터 전처리")
print("=" * 60)

# Pass만 필터링
passes = train_full[train_full['type_name'] == 'Pass'].copy()
print(f"\n📊 전체 Pass 액션: {len(passes):,}개")

# 패스 성공 여부
passes['is_success'] = (passes['result_name'] == 'Successful').astype(int)

# 패스 거리
passes['pass_distance'] = np.sqrt(
    (passes['end_x'] - passes['start_x'])**2 + 
    (passes['end_y'] - passes['start_y'])**2
)

# 전진 패스 (홈/원정 고려)
passes['forward_distance'] = np.where(
    passes['is_home'],
    passes['end_x'] - passes['start_x'],
    passes['start_x'] - passes['end_x']
)
passes['is_forward'] = (passes['forward_distance'] > 0).astype(int)

# 측면 패스
passes['is_wide'] = ((passes['start_y'] < 20) | (passes['start_y'] > 48)).astype(int)

print("✅ 패스 통계 변수 생성 완료")


Pass 데이터 전처리

📊 전체 Pass 액션: 178,582개
✅ 패스 통계 변수 생성 완료


## 3. 선수/팀 통계 계산

### 3.1 선수별 통계 계산 (경기별 누적)

In [7]:
print("\n" + "=" * 60)
print("선수별 통계 계산 (경기별 누적)")
print("=" * 60)

# game_id 정렬
game_ids = sorted(passes['game_id'].unique())
print(f"\n📊 총 경기 수: {len(game_ids)}개")

# 선수별 누적 통계 저장
player_cumulative_stats = {}

# 경기별로 순회하며 누적 통계 계산
for i, game_id in enumerate(tqdm(game_ids, desc="선수 통계 계산")):
    if i == 0:
        # 첫 경기: 통계 없음 (나중에 전체 평균 사용)
        player_cumulative_stats[game_id] = None
    else:
        # 이전 경기들의 데이터
        prev_games = game_ids[:i]
        prev_passes = passes[passes['game_id'].isin(prev_games)]
        
        # 선수별 통계 계산
        player_stats = prev_passes.groupby('player_id').agg({
            'pass_distance': 'mean',
            'is_forward': 'mean',
            'is_success': 'mean',
            'player_id': 'count'  # 패스 횟수
        }).rename(columns={'player_id': 'pass_count'})
        
        # dict로 변환 (쉽게 접근하기 위해)
        player_cumulative_stats[game_id] = player_stats.to_dict('index')

print(f"\n✅ 선수 통계 계산 완료")


선수별 통계 계산 (경기별 누적)

📊 총 경기 수: 198개


선수 통계 계산: 100%|██████████| 198/198 [00:02<00:00, 84.56it/s] 


✅ 선수 통계 계산 완료


In [8]:
# 전체 평균값 (신인 선수용)
global_player_stats = {
    'pass_distance': passes['pass_distance'].mean(),
    'is_forward': passes['is_forward'].mean(),
    'is_success': passes['is_success'].mean(),
    'pass_count': 50  # 기본값
}

print(f"\n🌍 전체 평균:")
print(f"   평균 패스 거리: {global_player_stats['pass_distance']:.2f}m")
print(f"   전진 패스 비율: {global_player_stats['is_forward']:.1%}")
print(f"   패스 성공률: {global_player_stats['is_success']:.1%}")


🌍 전체 평균:
   평균 패스 거리: 16.84m
   전진 패스 비율: 49.9%
   패스 성공률: 86.3%


### 3.2 팀별 통계 계산

In [9]:
print("\n" + "=" * 60)
print("팀별 통계 계산 (경기별 누적)")
print("=" * 60)

# 팀별 누적 통계 저장
team_cumulative_stats = {}

for i, game_id in enumerate(tqdm(game_ids, desc="팀 통계 계산")):
    if i == 0:
        team_cumulative_stats[game_id] = None
    else:
        prev_games = game_ids[:i]
        prev_passes = passes[passes['game_id'].isin(prev_games)]
        
        # 팀별 통계
        team_stats = prev_passes.groupby('team_id').agg({
            'pass_distance': 'mean',
            'is_wide': 'mean'
        }).rename(columns={'is_wide': 'attack_style'})
        
        team_cumulative_stats[game_id] = team_stats.to_dict('index')

print(f"\n✅ 팀 통계 계산 완료")


팀별 통계 계산 (경기별 누적)


팀 통계 계산: 100%|██████████| 198/198 [00:02<00:00, 97.00it/s] 



✅ 팀 통계 계산 완료


In [10]:
# 전체 평균값
global_team_stats = {
    'pass_distance': passes['pass_distance'].mean(),
    'attack_style': passes['is_wide'].mean()
}

print(f"\n🌍 전체 평균:")
print(f"   평균 패스 거리: {global_team_stats['pass_distance']:.2f}m")
print(f"   측면 선호도: {global_team_stats['attack_style']:.1%}")


🌍 전체 평균:
   평균 패스 거리: 16.84m
   측면 선호도: 60.5%


## 4. Phase 3 데이터에 통계 추가

### 4.1 선수 통계 추가

In [11]:
print("\n" + "=" * 60)
print("Phase 3 데이터에 피처 추가")
print("=" * 60)

# 선수 통계 함수 (단순화)
def get_player_stat(row, stat_name):
    """해당 경기/선수의 통계 가져오기"""
    game_id = row['game_id']
    player_id = row['player_id']
    
    # 해당 경기의 통계
    game_stats = player_cumulative_stats.get(game_id)
    
    # 통계 없으면 전체 평균
    if game_stats is None:
        return global_player_stats[stat_name]
    
    # 선수 통계 없으면 전체 평균
    if player_id not in game_stats:
        return global_player_stats[stat_name]
    
    # 선수 통계 반환
    return game_stats[player_id][stat_name]

print("\n🏃 선수 통계 피처 생성 중...")
df_v3['player_avg_pass_distance'] = df_v3.apply(
    lambda row: get_player_stat(row, 'pass_distance'), axis=1
)
df_v3['player_forward_ratio'] = df_v3.apply(
    lambda row: get_player_stat(row, 'is_forward'), axis=1
)
df_v3['player_success_rate'] = df_v3.apply(
    lambda row: get_player_stat(row, 'is_success'), axis=1
)
df_v3['player_pass_count'] = df_v3.apply(
    lambda row: get_player_stat(row, 'pass_count'), axis=1
)

print("✅ 선수 통계 피처 추가 완료")


Phase 3 데이터에 피처 추가

🏃 선수 통계 피처 생성 중...
✅ 선수 통계 피처 추가 완료


In [12]:
# 검증 (즉시)
print("\n📊 선수 통계 검증:")
print(f"   player_avg_pass_distance: {df_v3['player_avg_pass_distance'].nunique()}개 고유값")
print(f"   player_forward_ratio: {df_v3['player_forward_ratio'].nunique()}개 고유값")
print(f"   player_success_rate: {df_v3['player_success_rate'].nunique()}개 고유값")
print(f"   player_pass_count: {df_v3['player_pass_count'].nunique()}개 고유값")


📊 선수 통계 검증:
   player_avg_pass_distance: 4755개 고유값
   player_forward_ratio: 3178개 고유값
   player_success_rate: 3190개 고유값
   player_pass_count: 1102개 고유값


### 4.2 팀별 통계 추가

In [13]:
def get_team_stat(row, stat_name):
    """해당 경기/팀의 통계 가져오기"""
    game_id = row['game_id']
    team_id = row['team_id']
    
    game_stats = team_cumulative_stats.get(game_id)
    
    if game_stats is None:
        return global_team_stats[stat_name]
    
    if team_id not in game_stats:
        return global_team_stats[stat_name]
    
    return game_stats[team_id][stat_name]

print("\n⚽ 팀 통계 피처 생성 중...")
df_v3['team_avg_pass_distance'] = df_v3.apply(
    lambda row: get_team_stat(row, 'pass_distance'), axis=1
)
df_v3['team_attack_style'] = df_v3.apply(
    lambda row: get_team_stat(row, 'attack_style'), axis=1
)

print("✅ 팀 통계 피처 추가 완료")



⚽ 팀 통계 피처 생성 중...
✅ 팀 통계 피처 추가 완료


In [14]:
# 검증
print("\n📊 팀 통계 검증:")
print(f"   team_avg_pass_distance: {df_v3['team_avg_pass_distance'].nunique()}개 고유값")
print(f"   team_attack_style: {df_v3['team_attack_style'].nunique()}개 고유값")


📊 팀 통계 검증:
   team_avg_pass_distance: 385개 고유값
   team_attack_style: 385개 고유값


### 4.3 경기 흐름 피처

In [15]:
print("\n⏱️ 경기 흐름 피처 생성 중...")

# match_info에서 득점 정보 가져오기
score_info = match_info[['game_id', 'home_score', 'away_score']].copy()

df_v3 = df_v3.merge(score_info, on='game_id', how='left')

# 득점차 계산 (내 팀 - 상대 팀)
df_v3['score_diff'] = np.where(
    df_v3['is_home'],
    df_v3['home_score'] - df_v3['away_score'],
    df_v3['away_score'] - df_v3['home_score']
)

# 경기 진행률 (0~1)
df_v3['match_period_normalized'] = df_v3['time_seconds'] / 5400  # 90분 = 5400초

# 후반 75분 이후 (4050초 이후)
df_v3['is_late_game'] = (df_v3['time_seconds'] >= 4050).astype(int)

print("✅ 경기 흐름 피처 추가 완료")


⏱️ 경기 흐름 피처 생성 중...
✅ 경기 흐름 피처 추가 완료


## 5. 최종 검증 및 저장

In [16]:
print("\n" + "=" * 60)
print("최종 검증")
print("=" * 60)

# Phase 4 피처
phase4_cols = [
    'player_avg_pass_distance', 'player_forward_ratio', 
    'player_success_rate', 'player_pass_count',
    'team_avg_pass_distance', 'team_attack_style',
    'score_diff', 'match_period_normalized', 'is_late_game'
]

print("\n📊 Phase 4 피처 다양성:")
for col in phase4_cols:
    nunique = df_v3[col].nunique()
    print(f"   {col:30s}: {nunique:,}개 고유값")
    
    # 🚨 경고
    if nunique == 1:
        print(f"      ⚠️  WARNING: 단 1개 값만 존재!")


최종 검증

📊 Phase 4 피처 다양성:
   player_avg_pass_distance      : 4,755개 고유값
   player_forward_ratio          : 3,178개 고유값
   player_success_rate           : 3,190개 고유값
   player_pass_count             : 1,102개 고유값
   team_avg_pass_distance        : 385개 고유값
   team_attack_style             : 385개 고유값
   score_diff                    : 13개 고유값
   match_period_normalized       : 14,784개 고유값
   is_late_game                  : 1개 고유값
      ⚠️  WARNING: 단 1개 값만 존재!


In [17]:
# 결측치
print(f"\n📊 결측치:")
missing = df_v3[phase4_cols].isnull().sum()
missing = missing[missing > 0]
if len(missing) > 0:
    print(missing)
else:
    print("   ✅ 없음")


📊 결측치:
   ✅ 없음


In [18]:
# 극단값
print(f"\n📊 Phase 4 피처 범위:")
for col in phase4_cols:
    print(f"   {col:30s}: [{df_v3[col].min():.2f}, {df_v3[col].max():.2f}]")


📊 Phase 4 피처 범위:
   player_avg_pass_distance      : [4.04, 55.26]
   player_forward_ratio          : [0.00, 1.00]
   player_success_rate           : [0.00, 1.00]
   player_pass_count             : [1.00, 1874.00]
   team_avg_pass_distance        : [15.41, 18.17]
   team_attack_style             : [0.54, 0.68]
   score_diff                    : [-6.00, 6.00]
   match_period_normalized       : [0.00, 0.67]
   is_late_game                  : [0.00, 0.00]


## 6. 저장

In [19]:
# 불필요한 컬럼 제거
cols_to_drop = ['game_id', 'team_id', 'player_id', 'is_home', 'home_score', 'away_score']
df_v4 = df_v3.drop(columns=cols_to_drop)

print(f"\n✅ 최종 컬럼: {len(df_v4.columns)}개")

# 저장
output_path = PROCESSED_DIR / 'train_final_passes_v4.csv'
df_v4.to_csv(output_path, index=False)

print(f"\n💾 저장 완료: {output_path}")
print(f"   Shape: {df_v4.shape}")

print("\n" + "=" * 60)
print("Phase 4 피처 생성 완료! 🎉")
print("=" * 60)


✅ 최종 컬럼: 51개

💾 저장 완료: ..\data\processed\train_final_passes_v4.csv
   Shape: (15435, 51)

Phase 4 피처 생성 완료! 🎉


In [20]:
# 샘플 확인
print("\n📋 선수 통계 샘플 (10개):")
sample_cols = ['player_avg_pass_distance', 'player_forward_ratio', 
               'player_success_rate', 'player_pass_count']
print(df_v4[sample_cols].head(10))


📋 선수 통계 샘플 (10개):
   player_avg_pass_distance  player_forward_ratio  player_success_rate  \
0                 16.841951              0.499166             0.863441   
1                 16.841951              0.499166             0.863441   
2                 16.841951              0.499166             0.863441   
3                 16.841951              0.499166             0.863441   
4                 16.841951              0.499166             0.863441   
5                 16.841951              0.499166             0.863441   
6                 16.841951              0.499166             0.863441   
7                 16.841951              0.499166             0.863441   
8                 16.841951              0.499166             0.863441   
9                 16.841951              0.499166             0.863441   

   player_pass_count  
0                 50  
1                 50  
2                 50  
3                 50  
4                 50  
5                 50  
6    

## 7. Phase 4 데이터 검증 및 문제 진단

In [21]:
# 생성된 데이터 load
df_v4 = pd.read_csv(PROCESSED_DIR / 'train_final_passes_v4.csv')

### 7.1 기본 정보

In [22]:
print(f"\n✅ Shape: {df_v4.shape}")
print(f"✅ 컬럼 수: {len(df_v4.columns)}")


✅ Shape: (15435, 51)
✅ 컬럼 수: 51


### 7.2 Phase 4 피처 상세 분석

In [23]:
phase4_cols = [
    'player_avg_pass_distance', 'player_forward_ratio', 
    'player_success_rate', 'player_pass_count',
    'team_avg_pass_distance', 'team_attack_style',
    'score_diff', 'match_period_normalized', 'is_late_game'
]

print("\n" + "=" * 60)
print("Phase 4 피처 상세 분석")
print("=" * 60)

for col in phase4_cols:
    print(f"\n📊 {col}")
    print(f"   Min:    {df_v4[col].min():.4f}")
    print(f"   Max:    {df_v4[col].max():.4f}")
    print(f"   Mean:   {df_v4[col].mean():.4f}")
    print(f"   Std:    {df_v4[col].std():.4f}")
    print(f"   Unique: {df_v4[col].nunique():,}개")
    
    # 🚨 경고: 표준편차가 0이면 문제!
    if df_v4[col].std() < 0.01:
        print(f"   ⚠️  WARNING: 거의 변화 없음! (Std < 0.01)")


Phase 4 피처 상세 분석

📊 player_avg_pass_distance
   Min:    4.0377
   Max:    55.2554
   Mean:   16.4022
   Std:    4.0992
   Unique: 4,755개

📊 player_forward_ratio
   Min:    0.0000
   Max:    1.0000
   Mean:   0.4996
   Std:    0.0776
   Unique: 3,178개

📊 player_success_rate
   Min:    0.0000
   Max:    1.0000
   Mean:   0.8463
   Std:    0.0703
   Unique: 3,190개

📊 player_pass_count
   Min:    1.0000
   Max:    1874.0000
   Mean:   359.2643
   Std:    338.0036
   Unique: 1,102개

📊 team_avg_pass_distance
   Min:    15.4121
   Max:    18.1666
   Mean:   16.8435
   Std:    0.4836
   Unique: 385개

📊 team_attack_style
   Min:    0.5442
   Max:    0.6823
   Mean:   0.6107
   Std:    0.0226
   Unique: 385개

📊 score_diff
   Min:    -6.0000
   Max:    6.0000
   Mean:   -0.0152
   Std:    1.6305
   Unique: 13개

📊 match_period_normalized
   Min:    0.0008
   Max:    0.6693
   Mean:   0.2826
   Std:    0.1699
   Unique: 14,664개

📊 is_late_game
   Min:    0.0000
   Max:    0.0000
   Mean:   0.0000


### 7.3 선수별 다양성 확인

In [24]:
print("\n" + "=" * 60)
print("선수별 통계 다양성 확인")
print("=" * 60)

# 선수 통계 샘플 확인
print("\n📋 선수 통계 샘플 (10개):")
sample_cols = ['player_avg_pass_distance', 'player_forward_ratio', 
               'player_success_rate', 'player_pass_count']
print(df_v4[sample_cols].head(10))

# 고유값 확인
print("\n📊 고유값 개수:")
for col in sample_cols:
    print(f"   {col:30s}: {df_v4[col].nunique():,}개")


선수별 통계 다양성 확인

📋 선수 통계 샘플 (10개):
   player_avg_pass_distance  player_forward_ratio  player_success_rate  \
0                 16.841951              0.499166             0.863441   
1                 16.841951              0.499166             0.863441   
2                 16.841951              0.499166             0.863441   
3                 16.841951              0.499166             0.863441   
4                 16.841951              0.499166             0.863441   
5                 16.841951              0.499166             0.863441   
6                 16.841951              0.499166             0.863441   
7                 16.841951              0.499166             0.863441   
8                 16.841951              0.499166             0.863441   
9                 16.841951              0.499166             0.863441   

   player_pass_count  
0                 50  
1                 50  
2                 50  
3                 50  
4                 50  
5            

In [25]:
# 🚨 문제 진단
print("\n" + "=" * 60)
print("문제 진단")
print("=" * 60)

issues = []


문제 진단


In [26]:
# 체크 1: 선수 통계가 모두 동일한가?
if df_v4['player_avg_pass_distance'].nunique() == 1:
    issues.append("❌ 선수별 평균 패스 거리가 모두 동일 (전체 평균만 사용됨)")
else:
    print("✅ 선수별 평균 패스 거리: 다양성 있음")

if df_v4['player_forward_ratio'].nunique() == 1:
    issues.append("❌ 선수별 전진 패스 비율이 모두 동일")
else:
    print("✅ 선수별 전진 패스 비율: 다양성 있음")

if df_v4['player_success_rate'].nunique() == 1:
    issues.append("❌ 선수별 성공률이 모두 동일")
else:
    print("✅ 선수별 성공률: 다양성 있음")

if df_v4['player_pass_count'].nunique() == 1:
    issues.append("❌ 선수별 패스 횟수가 모두 10 (기본값만 사용됨)")
else:
    print("✅ 선수별 패스 횟수: 다양성 있음")

✅ 선수별 평균 패스 거리: 다양성 있음
✅ 선수별 전진 패스 비율: 다양성 있음
✅ 선수별 성공률: 다양성 있음
✅ 선수별 패스 횟수: 다양성 있음


In [27]:
# 체크 2: 팀 통계
if df_v4['team_avg_pass_distance'].nunique() == 1:
    issues.append("❌ 팀별 평균 패스 거리가 모두 동일")
else:
    print("✅ 팀별 평균 패스 거리: 다양성 있음")

if df_v4['team_attack_style'].nunique() == 1:
    issues.append("❌ 팀별 공격 스타일이 모두 동일")
else:
    print("✅ 팀별 공격 스타일: 다양성 있음")

✅ 팀별 평균 패스 거리: 다양성 있음
✅ 팀별 공격 스타일: 다양성 있음


In [28]:
# 체크 3: 경기 흐름 (이건 정상일 수 있음)
if df_v4['score_diff'].nunique() == 1:
    issues.append("⚠️  득점차가 모두 동일 (0-0 경기만?)")
else:
    print(f"✅ 득점차: {df_v4['score_diff'].nunique()}개 고유값")

if df_v4['is_late_game'].nunique() == 1 and df_v4['is_late_game'].iloc[0] == 0:
    print("⚠️  후반 75분 이후 데이터 없음 (정상일 수 있음)")
else:
    print(f"✅ 후반전 데이터: 있음")

✅ 득점차: 13개 고유값
⚠️  후반 75분 이후 데이터 없음 (정상일 수 있음)


### 7.4  최종 판정

In [29]:
print("\n" + "=" * 60)
print("최종 판정")
print("=" * 60)

if len(issues) > 0:
    print("\n🚨 문제 발견:")
    for issue in issues:
        print(f"   {issue}")
    print("\n→ Phase 4 피처가 제대로 생성되지 않았습니다!")
    print("→ 코드 수정이 필요합니다.")
else:
    print("\n✅ 모든 검증 통과!")
    print("→ Phase 4 피처가 정상적으로 생성되었습니다.")


최종 판정

✅ 모든 검증 통과!
→ Phase 4 피처가 정상적으로 생성되었습니다.


### 8.5 추가 진단 정보

In [30]:
print("\n" + "=" * 60)
print("추가 진단 정보")
print("=" * 60)

# game_episode 확인
print(f"\n📋 game_episode 샘플:")
print(df_v4['game_episode'].head(5).tolist())

print(f"\n📊 고유 game_episode 수: {df_v4['game_episode'].nunique():,}개")
print(f"   (총 샘플: {len(df_v4):,}개)")


추가 진단 정보

📋 game_episode 샘플:
['126283_1', '126283_10', '126283_11', '126283_12', '126283_13']

📊 고유 game_episode 수: 15,435개
   (총 샘플: 15,435개)


In [31]:
# 상관관계 확인 (선수 통계끼리)
print(f"\n📊 선수 통계 간 상관관계:")
corr = df_v4[sample_cols].corr()
print(corr)

print("\n💡 해석:")
if corr.iloc[0, 1] > 0.99:  # 첫 번째와 두 번째 피처 상관관계
    print("   ⚠️  상관관계가 거의 1 → 모든 값이 동일할 가능성")
else:
    print("   ✅ 상관관계 정상 범위")


📊 선수 통계 간 상관관계:
                          player_avg_pass_distance  player_forward_ratio  \
player_avg_pass_distance                  1.000000              0.054221   
player_forward_ratio                      0.054221              1.000000   
player_success_rate                      -0.084481              0.016672   
player_pass_count                         0.125876              0.028800   

                          player_success_rate  player_pass_count  
player_avg_pass_distance            -0.084481           0.125876  
player_forward_ratio                 0.016672           0.028800  
player_success_rate                  1.000000           0.280146  
player_pass_count                    0.280146           1.000000  

💡 해석:
   ✅ 상관관계 정상 범위
